Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [19]:
from random import random
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
import numpy as np
import time

In [20]:
PROBLEM_SIZE = 20
NUM_SETS = 30
SETS = tuple(
    np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)]) 
    for _ in range(NUM_SETS)
)
State = namedtuple("State", ["taken", "not_taken"])

In [21]:
# function that validate the solution of the problemS
def goal_check(state):
    return np.all(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

In [22]:
# ensures the problem is solvable
while not goal_check(State(set(range(NUM_SETS)), set())):
    SETS = tuple(
        np.array([random() < 0.3 for _ in range(PROBLEM_SIZE)]) 
        for _ in range(NUM_SETS)
    )

In [23]:
# function that prints sets in a human readable way
def print_sets(sets_indexes):
    for index in sets_indexes:
        set = SETS[index]
        print(index, end=':\t')
        for element in set:
            if element:
                print('* ', end='')
            else: 
                print('_ ', end='')
        print()

# Differences from the solution proposed during the lecture

> Starting point: https://github.com/squillero/computational-intelligence/blob/95b17233025786bc4b6eedafee1718fb557487f1/2023-24/set-covering.ipynb 


## Cost function
I defined a cost function that was previously missing that returns how many sets are taken in a state. In this way the algorithm will try to find the solution with the least amount of sets involved.

## Heuristic
The heuristic is the same proposed by the professor, so the number of uncovered elements. This allows the algorithm to check first the sets with the most covered elements, because the priority queue takes the lowest value first.

In [24]:
# cost function for the astar search that returns the number of sets taken
def cost(state):
    return len(state.taken)

def heuristic1(state):
    return PROBLEM_SIZE - sum(
        reduce(
            np.logical_or,
            [SETS[i] for i in state.taken],
            np.array([False for _ in range(PROBLEM_SIZE)]),
        )
    )

def priority1(state):
    return cost(state) + heuristic1(state)



In [25]:
def astar_search(priority):
    state = State(set(), set(range(NUM_SETS)))
    frontier = PriorityQueue()
    frontier.put((priority(state), State(set(), set(range(NUM_SETS)))))

    total_time = 0
    step = 0
    _, current_state = frontier.get()
    while not goal_check(current_state):
        step += 1
        start_time = time.time()
        for action in current_state[1]:
            new_state = State(
                current_state.taken ^ {action}, 
                current_state.not_taken ^ {action}
            )
            frontier.put((priority(new_state), new_state))
        end_time = time.time()
        total_time += end_time - start_time
        print(f"Step {step:,} took {end_time - start_time:.6f} seconds")
        _, current_state = frontier.get()

    print(f"Solved in {step:,} steps and {total_time} seconds\n")
    return current_state.taken

In [26]:
astar_solution1 = astar_search(priority1)

Step 1 took 0.000000 seconds
Step 2 took 0.000000 seconds
Step 3 took 0.000000 seconds
Step 4 took 0.000000 seconds
Step 5 took 0.000000 seconds
Solved in 5 steps and 0.0 seconds



In [27]:
print("A* first solution:")
print(astar_solution1)
print_sets(astar_solution1)

A* first solution:
{2, 26, 18, 27}
2:	* * _ _ _ * _ _ * _ _ _ * * _ _ _ _ * _ 
26:	_ * * _ * _ _ * * * _ _ * _ * * * _ _ * 
18:	* _ * * _ _ _ _ * _ _ * * _ _ _ _ _ _ _ 
27:	_ _ _ * _ _ * _ _ * * _ * * _ * _ * _ _ 


# Results

